In [ ]:
# 네이버 자동 로그인하기

from selenium import webdriver
import time

#크롬 드라이버 설정 및 웹 페이지 열기
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

#사용자에게 필요한 정보들을를 입력 받습니다.
v_id = input('네이버 로그인 ID를 입력하세요:')
v_passwd = input('네이버 로그인 비밀번호를 입력하세요:')
time.sleep(2)

# 네이버 자동 로그인
driver.get("https://nid.naver.com/nidlogin.login")

driver.execute_script("document.getElementsByName('id')[0].value=\'"+v_id+"\'")
driver.execute_script("document.getElementsByName('pw')[0].value=\'"+v_passwd+"\'")
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(2)
    

In [ ]:
# headless mode 활용하기
# 대한민국 구석구석 사이트 상세 정보 수집하기

#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
import time          
import pandas as pd    
import math

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.정보를 수집할 키워드는 무엇입니까?: ')

cnt = int(input('2.몇 건의 정보를 수집할까요? :'))
page_cnt = math.ceil( cnt / 10 )
print('총 %s 건의 정보를 수집하기 위해 %s 페이지까지 이동할 예정입니다' %(cnt , page_cnt))

#Step 3. 수집된 데이터를 저장할 폴더 이름 입력받기 
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"
    
print("\n")    

#Step 3. 검색어 입력한 후 검색하여 해당 장르로 이동하기
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


#headless 모드 설정하기
options = webdriver.ChromeOptions()
options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-infobars")
driver = webdriver.Chrome(options=options)
#driver= webdriver.Chrome()

query_url = 'https://korean.visitkorea.or.kr/'

driver.get(query_url)
time.sleep(2)
driver.maximize_window()

s_time = time.time( )
element = driver.find_element_by_id("inp_search")
driver.find_element_by_id("inp_search").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

# Step 5. 본문 내역 추출하기
time.sleep(0.2)
s_time = time.time( ) 

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
org2=[]          # 지자체명 컬럼
content2=[ ]     # 본문 내용 컬럼
tag2=[ ]         # 해시태그 컬럼

no = 1           # 게시글 번호 초기값

for a in range(1,page_cnt+1) :
    print('')
    print('현재 %s 페이지에 있는 정보를 수집합니다' %a)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType type1').find_all('li')
    cnt_no = 1
    
    for b in content_list:
        
        try :
            title = b.find('div','area_txt').find('div','tit').get_text().strip()
        except :
            continue
        else :
            # 1. 게시글 번호
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.번호 : %s" %no)

            # 2. 게시글 제목
            title2.append(title)
            print("2.제목 : %s" %title )
            
            # 3. 지자체이름
            org = b.find('div','service').find('p').get_text().strip()
            org2.append(org)
            print("3.지자체이름: %s" %org)
            
            #4. 해시태그
            try :
                tag = b.find('p','tag_type').get_text().replace("#"," #")
            except :
                tag = ' '
                
            tag2.append(tag)
            print("4.해시태그: %s" %tag)
            
            #5. 본문
            
            driver.find_element_by_xpath('//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %cnt_no).click()
            time.sleep(5)
            
            html_2 = driver.page_source
            soup_2 = BeautifulSoup(html_2, 'html.parser')
            
            content_1 = soup_2.find('div','inr_wrap')
            content_2 = soup_2.find('div','box_txtPhoto')
            
            if content_1 :
                content = content_1.find('div','inr').find('p').get_text()
                content2.append(content)
                print('5.본문내용:', content)
            elif content_2 :
                content = soup_2.find('div','box_txtPhoto').get_text()
                content2.append(content)
                print('5.본문내용:', content)            
            
            driver.back()
            
            time.sleep(5)
                       
            no += 1         # 전체 게시글 번호용 값
                       
            if a == 1 and cnt_no == 4 : # 각 페이지별 게시글 번호용 값
                cnt_no += 2   
            else :
                cnt_no += 1    
            
            if no > cnt :
                 break
                
            time.sleep(0.5)
            
    a += 1
    try :
        driver.find_element_by_link_text('%s' %a).click()
    except :
        driver.find_element_by_link_text('다음').click()

    time.sleep(0.5)

# Step 6. 결과 저장하기
import time
import os

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt)

fc_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.csv'
fx_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.xls'

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['지자체명']=pd.Series(org2)
df['본문내용']=pd.Series(content2)
df['해시태그']=pd.Series(tag2)

# xls , csv 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print('xls 저장경로: ',fx_name)
print('csv 저장경로: ',fc_name)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)

driver.close( )